<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">In this project we will be investigating greenhouse gas emissions data across years, provinces and sectors. We will also be looking at specific facilities (and their respective industry) that have significantly high emissions.</span>

The aim is to identify trends across time and differences between provinces with regards to emission levels and emission-heavy sectors.

The greenhouse gas emissions data used in this project was taken from Canada's Official Greenhouse Gas Inventory: https://open.canada.ca/data/en/dataset/779c7bcf-4982-47eb-af1b-a33618a05e5b

The population data was taken from statistics canada: https://www.statcan.gc.ca/en/subjects-start/population\_and\_demography

The license for these datasets are the Open Government License: https://open.canada.ca/en/open-government-licence-canada

In [1]:
USE EmissionsDb;

Commands completed successfully.

Total execution time: 00:00:00.026

In [3]:
-- Which year did emissions peak in Canada?
SELECT TOP 10 Year, sum(mt_CO2eq) AS mt_CO2eq_sum
FROM EmissionsDb.dbo.GHG_emissions_by_sector
WHERE Region = 'Canada' 
AND Source = 'National Inventory Total'
GROUP BY  Year
ORDER BY mt_CO2eq_sum DESC;

(10 rows affected)

Total execution time: 00:00:00.049

Year,mt_CO2eq_sum
2007,748.0700868
2004,736.7254679
2003,734.1405466
2005,732.2187885
2008,730.8012349
2006,725.286361
2018,724.6149832
2019,723.6793027
2013,723.0963086
2015,722.9164176


From the table above, we can see that Canada's CO2eq emissions had a peak in 2007, but total emissions have been fairly constant for the last two decades. Let's look at the most recent year (2021) and determine how each indsutry contributed to the overall Canadian emissions that year.

In [4]:
SELECT Source AS sector, SUM(mt_CO2eq) AS emissions
FROM EmissionsDb.dbo.GHG_emissions_by_sector
WHERE Year = 2021 
AND Region = 'Canada' 
AND Sector IS NULL 
AND Source NOT LIKE 'Nat%'
GROUP BY Source
ORDER BY emissions DESC;

(9 rows affected)

Total execution time: 00:00:00.029

sector,emissions
Oil and Gas,189.1523515
Transport,150.1147371
Buildings,87.16960461
Heavy Industry,76.81255283
Agriculture,68.51682202
Electricity,51.67781389
"Light Manufacturing, Construction and Forest Resources",23.29859767
Waste,21.08342393
Coal Production,2.601783045


In 2021, the oil and gas industry accounted for the most emissions in Canada followed by the transport industry. The lowest contributors were coal production, waste and light manufacturing.

We'll now join on our population table to investigate the CO2 emissions per capita of each province in 2021. We'll look at emissions per capita in tons instead of megatons to keep our numbers at a good scale.

In [10]:
-- emissions per capita in each province
SELECT Province, (mt_CO2eq * 1000000 / Population) AS emissions_per_capita_tons
FROM GHG_emissions_by_sector AS em
    -- inner join on population table
    INNER JOIN canada_population AS pop
    ON em.Year = pop.Year 
    AND em.Region = pop.Province
WHERE em.Year = 2021
    -- only using provincial inventory total emissions
AND Source LIKE '%Total'
ORDER BY emissions_per_capita_tons DESC;

(13 rows affected)

Total execution time: 00:00:00.031

Province,emissions_per_capita_tons
Alberta,57.64218901370524
Saskatchewan,56.79830610084021
Canada,17.53829729838187
Newfoundland and Labrador,16.016960714917037
Nunavut,15.762852433834455
Yukon,15.030616462427746
New Brunswick,15.016297928385447
Manitoba,14.872245342781751
Nova Scotia,14.730859323369494
British Columbia,11.424864554248078


From the table above, we can see that Alberta had the highest emissions per capita in 2021, followed by Saskatchewan. These provinces had higher emissions per capita than Canada overall. Prince Edward Island and Quebec had the lowest emissions per capita.

Let's look at the highest emitting sectors in each province now.

In [77]:
-- outer query joins original table with subquery to match sector names with highest-emitting sectors
SELECT highest_sector.Region, ebs.Source AS Sector, highest_sector.max_emissions AS [Emissions(MT)]
FROM GHG_emissions_by_sector AS ebs
    INNER JOIN (
        -- this subquery returns each province and its emissions from highest-emitting sector but w/o sector names
        SELECT Region, MAX(mt_CO2eq) AS max_emissions
        FROM GHG_emissions_by_sector
        WHERE Year = '2021'
        -- to get only total emissions for sector
        AND Sector IS NULL
        -- don't want provincial total 
        AND Source NOT LIKE '%Total'
        GROUP BY Region) AS highest_sector
    ON ebs.Region = highest_sector.Region
    AND ebs.mt_CO2eq = highest_sector.max_emissions
ORDER BY Sector;

Warning: Null value is eliminated by an aggregate or other SET operation.

(14 rows affected)

Total execution time: 00:00:00.035

Region,Sector,Emissions(MT)
Manitoba,Agriculture,7.271065328
Nova Scotia,Electricity,6.084949956
New Brunswick,Oil and Gas,3.079905636
Saskatchewan,Oil and Gas,16.5757484
Alberta,Oil and Gas,144.7288326
Canada,Oil and Gas,189.1523515
Newfoundland and Labrador,Transport,3.494964056
Prince Edward Island,Transport,0.659413914
British Columbia,Transport,21.61338393
Yukon,Transport,0.358667976


In 2021, the highest emitting sectors in Alberta and Saskatchewan were Oil and Gas. For most other provinces and territories the transport sector had the highest emissions.

Let's now look at high emitting facilities across Canada. Facilities that emit over 10 kilotonnes of GHGs in CO2eq are mandated to report their emissions to Environment & Climate Change Canada.

What were the top 5 GHG emitting facilities in Canada in 2021 and where are they located?

In [84]:
-- Selecting top 5 emitting facilities in Canada
SELECT TOP 5 facility_name, municipality, province, total_emissions_tonnes_CO2e / 1000000 AS emissions
FROM facility_emissions
WHERE reference_year = 2021
ORDER BY emissions DESC;

(5 rows affected)

Total execution time: 00:00:00.025

facility_name,municipality,province,emissions
Mildred Lake and Aurora Plant Sites,Fort McMurray,Alberta,12.50201383
Suncor Energy Inc. Oil Sands,Fort McMurray,Alberta,8.030929245
Genesee Thermal Generating Station,Warburg,Alberta,7.5937774529999995
Cold Lake,Grande Centre,Alberta,5.771388332000001
Firebag,Ft. McMurray,Alberta,5.582564512


It is no surprise to see that the top emitting facilities in Canada are all in Alberta.

  

What is the top <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">emitting facility in each province and how much of its provinces emissions does each account for?</span>

In [98]:
 -- outer query joins facility, municipality names
 SELECT outside.facility_name, outside.province, outside.municipality, ROUND(inside.emissions, 2) AS emissions, 
 ROUND((inside.emissions / mt_CO2eq * 100), 2) AS percent_prov_emissions 
 FROM facility_emissions AS outside
    -- join adds facility names
    INNER JOIN (
        -- query returns emissions for most emitting facility in each province but no facility names
        SELECT province, MAX(total_emissions_tonnes_CO2e) / 1000000 AS emissions
        FROM facility_emissions
        WHERE reference_year = 2021
        GROUP BY province) AS inside
    ON outside.province = inside.province
    AND (outside.total_emissions_tonnes_CO2e) / 1000000 = inside.emissions
    -- join to retrieve total emissions for each province
    LEFT JOIN GHG_emissions_by_sector AS ges
    ON outside.province = ges.region
    AND ges.Year = 2021
    AND ges.Source = 'Provincial Inventory Total';

(13 rows affected)

Total execution time: 00:00:00.057

facility_name,province,municipality,emissions,percent_prov_emissions
Algoma Steel Inc,Ontario,Sault Ste. Marie,4.11,2.73
Boundary Dam Power Station,Saskatchewan,Estevan,4.29,6.39
Carol Project,Newfoundland and Labrador,Labrador City,0.89,10.68
Cavendish Farms,Prince Edward Island,New Annan,0.09,5.43
Lingan Generating Station,Nova Scotia,Lingan,2.64,18.07
Mildred Lake and Aurora Plant Sites,Alberta,Fort McMurray,12.5,4.88
Transmission Mainline - BC,British Columbia,Prince George,1.1,1.84
Irving Oil Refinery,New Brunswick,Saint John,3.06,25.79
"Koch Fertilizer Canada, ULC",Manitoba,Brandon,0.65,3.14
Diavik Diamond Mines (2012) Inc.,Northwest Territories,Yellowknife,0.19,NULL


The query result above shows the top emitting facility for each province, its emissions in 2021 in megatons of CO2eq, and its contribution to total provincial emissions. It is interesting to see provinces like New Brunswick and Nova Scotia whose highest emitting facilities account for about 26% and 18% of total provincial emissions respectively. This is unlike most of the larger provinces where the largest emitter accounts for a much smaller portion of total emissions.

Total emissions data is not available for the territories, resulting in NULL values in those rows.

Which reporting companies had the highest total emissions across their facilities in 2021?

In [109]:
-- identifying top 10 companies with highest emissions across all facilities in Canada
SELECT TOP 10 company_legal_name AS company, ROUND(SUM(total_emissions_tonnes_CO2e) / 1000000, 2) AS emissions, COUNT(*) as num_facilities
FROM facility_emissions
WHERE reference_year = 2021
GROUP BY company_legal_name
ORDER BY emissions DESC;

(10 rows affected)

Total execution time: 00:00:00.018

company,emissions,num_facilities
Canadian Natural Resources Limited,14.96,78
Suncor Energy Oil Sands Limited Partnership,14.03,3
Saskatchewan Power Corporation,13.25,10
Syncrude Canada Ltd.,12.5,1
Imperial Oil Resources Limited,8.87,2
Capital Power Generation Services Inc.,7.59,1
Cenovus FCCL Ltd.,7.38,2
Husky Oil Operations Limited,6.69,31
TransAlta Generation Partnership,6.31,3
Nova Scotia Power Incorporated,6.1,6


The companies with the largest emissions across all their facilities are mostly oil and gas companies. It is interesting to see that Canadian Natural Resources (CNL) and Suncor Energy both have similar total emissions but very different facility numbers. CNL has 78 facilities (compared to 3 for Suncor) which indicates that each CNL facility likely produces a lot less GHGs than each Suncor facility.